In [12]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import random
import math
import csv
from collections import Counter
import itertools
import copy
import os
import glob

def simulate_ER(n,prob, opq_prob, opq_Set,S, iteration):
    R_list = []
    leftover_list = []
    end_list = []
    opq_index = []
    num_choice = n + len(opq_Set)
    population = list(range(num_choice))
    weight = prob+opq_prob
    for t in range(iteration):
        inv = [S for i in range(n)]
        flag = 1
        while flag ==1:
            choice = random.choices(population, weights=weight, k=1)[0]
            if choice<n:
                inv[choice] = inv[choice]-1
            else:
                choice = opq_Set[choice-n]
                index_max = np.argmax([inv[index] for index in choice])
                inv[choice[index_max]] = inv[choice[index_max]] -1
                opq_index.append(index_max)
            if min(inv)==0:
                R_list.append(n*S-sum(inv))
                end_list.append(inv.index(min(inv)))
                leftover_list.append(inv.index(max(inv)))
                flag = 0      
    return(np.average(R_list), np.average([i**2 for i in R_list]), Counter(end_list), Counter(leftover_list), Counter(opq_index))

def cost(n,h,K,lamb, S, ER,ER2):
    holding = (2*n*S+1)*ER-ER2
    holding = float(h*holding)/(2*lamb*ER)
    ordering = float(K)/ER
    return(holding+ordering, holding, ordering)

# Parameters
n=3
K=10000
h=1
lamb=1
S = 50

# traditional selling, no opaque
q=0
prob = [float(1-q)/n for i in range(n)]
opq_Set = [[i for i in range(n)]]
opq_prob = [q]
[ER, ER2,  end_list0, leftover_list0 , opqindex_0]= simulate_ER(n,prob, opq_prob, opq_Set,S, 50000)
baseline1 = cost(n,h,K,lamb, S, ER,ER2)
print(baseline1[0])

### Three Scenarios ###

qi = [0.6, 0, 0] # Also use [0.2, 0.2, 0.2] or [0.3, 0.3, 0]
prob = [float(1)/(n)*(1- qi[i]) for i in range(n)]
opq_Set = [list(range(n))]
opq_prob = [float(1-sum(prob))]
[ER, ER2, end_list2, leftover_list2, opqindex_2] = simulate_ER(n,prob, opq_prob, opq_Set,S, 50000)
result = cost(n,h,K,lamb, S, ER,ER2)[0]
    
# Generate data in Table 1

print("Opaque Selling Allocation:")
print(opqindex_2[0]/sum(opqindex_2.values()) *100)
print(opqindex_2[1]/sum(opqindex_2.values()) *100)
print(opqindex_2[2]/sum(opqindex_2.values()) *100)

print("Out of Stock:")
print(end_list2[0]/sum(end_list2.values())*100)
print(end_list2[1]/sum(end_list2.values())*100)
print(end_list2[2]/sum(end_list2.values())*100)

print("Highest Leftover Inventory at Replenishment:")
print(leftover_list2[0]/sum(leftover_list2.values())*100)
print(leftover_list2[1]/sum(leftover_list2.values())*100)
print(leftover_list2[2]/sum(leftover_list2.values())*100)

print("Relative Savings: " + str(100*(1-result/baseline1[0]) ))

159.56594973241124
Opaque Selling Allocation:
82.09078490838172
9.145743905705064
8.763471185913213
Out of Stock:
1.272
50.322
48.406
Highest Leftover Inventory at Replenishment:
89.378
5.486
5.136
Relative Savings: 0.7869261767857627
